In [1]:
import os
import requests
from bs4 import BeautifulSoup as bs

In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import time 
import pandas as pd
from io import StringIO

In [5]:
url = "https://www.tankauction.com/"
payload = 
header= {'client_id':'yjj7479', 'password':'xodzmdhrtus0911!'}
r = requests.post(url,data = payload)
print(r.url, r.status_code)
print(r.text)

https://www.tankauction.com/inc/BgItems.php 200

<html lang="ko">
<head>
	<meta charset="utf-8" />
	<meta name="Generator" content="origin">
	<meta http-equiv="Content-Script-Type" content="text/javascript">
	<meta http-equiv="Content-Style-Type" content="text/css">
	<meta http-equiv="X-UA-Compatible" content="IE=edge">
	<meta name="viewport" content="width=1000px, maximum-scale=1.0">
		<title>탱크옥션</title>
	<meta name="Description" content="법원경매,부동산경매">
		<link rel="stylesheet" type="text/css" href="/css/font.css?ver=20241029">
	<link rel="stylesheet" type="text/css" href="/css/tank.css?ver=20241029">
	<link rel="stylesheet" type="text/css" href="/css/style.css?ver=20241029">
	<link rel="stylesheet" type="text/css" href="/css/button.css?ver=20241029">
			<link rel="stylesheet" type="text/css" href="/css/viewMob.css?ver=20241029">
		
	<link rel="stylesheet" type="text/css" href="/css/input.css?ver=20241029">
	<link rel="stylesheet" type="text/css" href="/css/tab.css?ver=20241029">
	<lin

In [30]:
options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")


driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
        )


driver = webdriver.Chrome()

driver.get('https://www.tankauction.com/')

wait = WebDriverWait(driver, 10)

# 로그인 버튼 선택
button = wait.until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, '.hand')
    ))
button.click()

driver.find_element(By.CSS_SELECTOR, '.hand').send_keys('yjj7479')



# 로그인 폼 채우기
driver.find_element(By.ID, 'client_id').send_keys('yjj7479')
driver.find_element(By.ID, 'password').send_keys('xodzmdhrtus0911!')
driver.find_element(By.ID, "button btn_white bigrounded").click()

time.sleep(5)  # 로그인 처리 대기

# 보호된 페이지 접근
driver.get('https://www.tankauction.com/inc/BgItems.php?title=%EC%95%84%ED%8C%8C%ED%8A%B8&t=a1&tid=2276168&dong=+52+&tg=A&tp=1&a=1&b=1&c=1&pnu=1132010500&addr=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EB%8F%84%EB%B4%89%EA%B5%AC+%EC%8C%8D%EB%AC%B8%EB%8F%99&bg=59-3&bldgNm=%ED%95%9C%EC%96%915&bldgSqm=65.37&bldgNmCnt=1&bldgDj=%ED%95%9C%EC%96%915&roadAddr=%ED%95%B4%EB%93%B1%EB%A1%9C25%EA%B8%B8+36&gb=1&tile_mode=&area=&search_mode=&class=&fromYear=2006&toYear=2025&fromMonth=1&toMonth=4&dealType=1&houseType=&landuse=&zoning=&oflanduse=&buildinguse=&area_gp=&pageNo=1&dataSize=20&pageSize=10&BBL=1&SBL=1&DBL=1')
html = driver.page_source

print(html)

driver.quit()

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (1070, 50)
  (Session info: chrome=135.0.7049.85)
Stacktrace:
	GetHandleVerifier [0x00007FF74AAE5355+78597]
	GetHandleVerifier [0x00007FF74AAE53B0+78688]
	(No symbol) [0x00007FF74A8991AA]
	(No symbol) [0x00007FF74A8F7089]
	(No symbol) [0x00007FF74A8F4A42]
	(No symbol) [0x00007FF74A8F1AE1]
	(No symbol) [0x00007FF74A8F0A42]
	(No symbol) [0x00007FF74A8E2204]
	(No symbol) [0x00007FF74A9170EA]
	(No symbol) [0x00007FF74A8E1AB6]
	(No symbol) [0x00007FF74A917300]
	(No symbol) [0x00007FF74A93F2BB]
	(No symbol) [0x00007FF74A916EC3]
	(No symbol) [0x00007FF74A8E03F8]
	(No symbol) [0x00007FF74A8E1163]
	GetHandleVerifier [0x00007FF74AD8EF0D+2870973]
	GetHandleVerifier [0x00007FF74AD896B8+2848360]
	GetHandleVerifier [0x00007FF74ADA6993+2967875]
	GetHandleVerifier [0x00007FF74AB0019A+188746]
	GetHandleVerifier [0x00007FF74AB0847F+222255]
	GetHandleVerifier [0x00007FF74AAED2D4+111236]
	GetHandleVerif

In [35]:
keyword1 = input("지역을 입력해주세요")
keyword2 = input("주거용과 아파트 중 하나를 입력해주세요")

options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")


driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
        )

driver.get('https://www.tankauction.com/')

wait = WebDriverWait(driver, 10)
# 로그인 버튼 선택
button = wait.until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, '.hand')
    ))
button.click()

# 로그인 폼 채우기
id_input = wait.until(EC.element_to_be_clickable(
        (By.ID, 'client_id')
    ))
id_input.send_keys('yjj7479')

driver.find_element(By.ID, 'passwd').send_keys('xodzmdhrtus0911!')
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, ".button.btn_white.bigrounded").click()

time.sleep(3)  # 로그인 처리 대기
# 경매 검색 
btn_search = wait.until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, ".menuicon_10")
    ))
btn_search.click()

# 지역 선택
siCd_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "siCd"))))
siCd_dropdown.select_by_visible_text(keyword1)

지역을 입력해주세요서울


In [20]:


# 로그인 폼 채우기
driver.find_element(By.ID, 'client_id').send_keys('yjj7479')
time.sleep(2)
driver.find_element(By.ID, 'passwd').send_keys('xodzmdhrtus0911!')
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, ".button.btn_white.bigrounded").click()

time.sleep(5)  # 로그인 처리 대기


In [34]:
keyword1 = input("지역을 입력해주세요")
keyword2 = input("주거용과 아파트 중 하나를 입력해주세요")

# 경매 검색 
btn_search = wait.until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, ".menuicon_10")
    ))
btn_search.click()
# 지역 선택
siCd_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "siCd"))))
siCd_dropdown.select_by_visible_text(keyword1)

# 물건종류 선택
siCd_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "ctgr"))))
siCd_dropdown.select_by_visible_text(keyword2)

# 물건종류 선택
siCd_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "minbAmtEnd"))))
siCd_dropdown.select_by_visible_text(keyword2)

지역을 입력해주세요서울


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00C88073+60707]
	GetHandleVerifier [0x00C880B4+60772]
	(No symbol) [0x00AB0683]
	(No symbol) [0x00AF8660]
	(No symbol) [0x00AF89FB]
	(No symbol) [0x00B41022]
	(No symbol) [0x00B1D094]
	(No symbol) [0x00B3E824]
	(No symbol) [0x00B1CE46]
	(No symbol) [0x00AEC5D3]
	(No symbol) [0x00AED424]
	GetHandleVerifier [0x00ECBB53+2435075]
	GetHandleVerifier [0x00EC70F3+2416035]
	GetHandleVerifier [0x00EE349C+2531660]
	GetHandleVerifier [0x00C9F145+155125]
	GetHandleVerifier [0x00CA5AED+182173]
	GetHandleVerifier [0x00C8F948+91640]
	GetHandleVerifier [0x00C8FAF0+92064]
	GetHandleVerifier [0x00C7A5B0+4704]
	BaseThreadInitThunk [0x76FD5D49+25]
	RtlInitializeExceptionChain [0x77BFCF0B+107]
	RtlGetAppContainerNamedObjectPath [0x77BFCE91+561]


ModuleNotFoundError: No module named 'PyQt5'

# 나중에 셀렉트 실행해볼것

In [ ]:
## Ex 5-5. QComboBox.

import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QComboBox


class MyApp(QWidget):

    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.lbl = QLabel('Option1', self)
        self.lbl.move(50, 150)

        cb = QComboBox(self)
        cb.addItem('Option1')
        cb.addItem('Option2')
        cb.addItem('Option3')
        cb.addItem('Option4')
        cb.move(50, 50)

        cb.activated[str].connect(self.onActivated)

        self.setWindowTitle('QComboBox')
        self.setGeometry(300, 300, 300, 200)
        self.show()

    def onActivated(self, text):
        self.lbl.setText(text)
        self.lbl.adjustSize()


if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = MyApp()
    sys.exit(app.exec_())